In [ ]:
import duckdb
import pandas as pd
import pyarrow.parquet as pq
from itertools import islice

# Connect to the DuckDB database
conn = duckdb.connect(':memory:')

In [ ]:
d = "/home/anadirov/Documents/big.parquet"
# pd_df = pd.read_parquet(d)

# q = f"""
#     create table data as select * from "{d}"
# """.strip()

In [ ]:
df = duckdb.read_parquet(d)

In [ ]:
parquet_file = pq.ParquetFile(d)
gen = parquet_file.iter_batches(batch_size=1000)

In [ ]:
def nth_item(generator, n):
    return next(islice(generator, n, n+1), None)

In [56]:
cv = pd.read_csv("/home/anadirov/Documents/kvi_table.csv", chunksize=5)

In [ ]:
# duckdb.from_df(i.to_pandas()).query("data", "select ProductName from  data")